In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!uv pip install keras_tuner efficientnet_pytorch optuna torch torchvision

Using Python 3.11.13 environment at: /usr
Resolved 53 packages in 576ms
   Building efficientnet-pytorch==0.7.1
   Building efficientnet-pytorch==0.7.1
⠙ Preparing packages... (0/17)
   Building efficientnet-pytorch==0.7.1
⠙ Preparing packages... (0/17)
alembic    ------------------------------     0 B/241.24 KiB
   Building efficientnet-pytorch==0.7.1
⠙ Preparing packages... (0/17)
alembic    ------------------------------     0 B/241.24 KiB
nvidia-cuda-cupti-cu12 ------------------------------     0 B/13.17 MiB
   Building efficientnet-pytorch==0.7.1
⠙ Preparing packages... (0/17)
alembic    ------------------------------ 16.00 KiB/241.24 KiB
nvidia-cuda-cupti-cu12 ------------------------------     0 B/13.17 MiB
   Building efficientnet-pytorch==0.7.1
⠙ Preparing packages... (0/17)
alembic    ------------------------------ 16.00 KiB/241.24 KiB
nvidia-cuda-cupti-cu12 ------------------------------     0 B/13.17 MiB
   Building efficientnet-pytorch==0.7.1
⠙ Preparing packages... (0/17

In [ ]:
import os
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from efficientnet_pytorch import EfficientNet
import optuna
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set xray_data_path
xray_data_path = '/content/drive/My Drive/xray_data/data'
print(f"xray_data_path set to: {xray_data_path}")

# Verify data existence
for subfolder in ['01', '02', '03']:
    subfolder_path = os.path.join(xray_data_path, subfolder)
    if not os.path.exists(subfolder_path):
        raise FileNotFoundError(f"Subfolder {subfolder_path} not found.")
    images = [f for f in os.listdir(subfolder_path) if f.endswith((".jpg", ".png", ".jpeg"))]
    print(f"{subfolder}: {len(images)} images found")
    if not images:
        raise FileNotFoundError(f"No images found in {subfolder_path}.")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define data transforms with balanced augmentation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((300, 300)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(15),
        transforms.RandomAffine(degrees=5, translate=(0.05, 0.05)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((300, 300)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((300, 300)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

# Split data into train, val, test (60-20-20)
base_dir = xray_data_path
classes = ["01", "02", "03"]
class_names = ["Class1", "Class2", "Class3"]
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")
test_dir = os.path.join(base_dir, "test")

# Create directories
for dir_path in [train_dir, val_dir, test_dir]:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_path, class_name), exist_ok=True)

# Split and copy images
for old_class, new_class in zip(classes, class_names):
    class_path = os.path.join(base_dir, old_class)
    images = [f for f in os.listdir(class_path) if f.endswith((".jpg", ".png", ".jpeg"))]
    image_paths = [os.path.join(class_path, img) for img in images]
    train_imgs, temp_imgs = train_test_split(image_paths, test_size=0.4, random_state=23)
    val_imgs, test_imgs = train_test_split(temp_imgs, test_size=0.5, random_state=23)

    for img in train_imgs:
        dest = os.path.join(train_dir, new_class, os.path.basename(img))
        shutil.copy(img, dest)
    for img in val_imgs:
        dest = os.path.join(val_dir, new_class, os.path.basename(img))
        shutil.copy(img, dest)
    for img in test_imgs:
        dest = os.path.join(test_dir, new_class, os.path.basename(img))
        shutil.copy(img, dest)

    print(f"Class {new_class}: Train={len(train_imgs)}, Val={len(val_imgs)}, Test={len(test_imgs)}")

# Verify files were copied
for dir_path, split_name in [(train_dir, "train"), (val_dir, "val"), (test_dir, "test")]:
    for class_name in class_names:
        class_dir = os.path.join(dir_path, class_name)
        num_files = len([f for f in os.listdir(class_dir) if f.endswith((".jpg", ".png", ".jpeg"))])
        print(f"{split_name}/{class_name}: {num_files} files")
        if num_files == 0:
            raise RuntimeError(f"No files found in {class_dir}. Copying failed.")
print("Data split complete!")

# Load datasets
train_dataset = ImageFolder(root=train_dir, transform=data_transforms['train'])
val_dataset = ImageFolder(root=val_dir, transform=data_transforms['val'])
test_dataset = ImageFolder(root=test_dir, transform=data_transforms['test'])

print(f"Train dataset size: {len(train_dataset)}")
print(f"Val dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")
print("Class mapping:", train_dataset.class_to_idx)

# Define the model with SiLU instead of Swish for TorchScript compatibility
def create_model(dropout_rate=0.35):
    model = EfficientNet.from_pretrained('efficientnet-b3')
    # Replace Swish with SiLU
    for module in model.modules():
        if hasattr(module, '_swish'):
            module._swish = nn.SiLU(inplace=True)
    num_features = model._fc.in_features
    model._fc = nn.Sequential(
        nn.Dropout(p=dropout_rate),
        nn.Linear(num_features, 3)
    )
    return model.to(device)

# Training function with early stopping and scheduler
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience=5):
    best_val_acc = 0.0
    patience_counter = 0
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        train_loss = running_loss / len(train_loader)

        model.eval()
        correct = 0
        total = 0
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(val_loader)
        val_acc = 100 * correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        scheduler.step(val_acc)  # Adjust LR based on val_acc

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_efficientnet.pth")
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    return best_val_acc

# Objective function for Optuna with updated suggestions
def objective(trial):
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)  # Updated
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    dropout_rate = trial.suggest_float("dropout_rate", 0.3, 0.5)  # Updated
    num_epochs = trial.suggest_int("num_epochs", 5, 15)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    model = create_model(dropout_rate)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=5e-5)

    val_acc = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)
    return val_acc

# Optimize hyperparameters with Optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)
print("Best hyperparameters:", study.best_params)
print("Best validation accuracy:", study.best_value)

# Train final model with best parameters
best_params = study.best_params
train_loader = DataLoader(train_dataset, batch_size=best_params["batch_size"], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=best_params["batch_size"], shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=best_params["batch_size"], shuffle=False)

final_model = create_model(best_params["dropout_rate"])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(final_model.parameters(), lr=best_params["lr"], weight_decay=5e-5)
train_model(final_model, train_loader, val_loader, criterion, optimizer, best_params["num_epochs"])


# Define the path to save the final model's state dictionary
# It's good practice to save it in your Google Drive if working in Colab
# so it persists after the Colab session ends.
final_model_save_path = '/content/drive/My Drive/xray_data/efficientnet_final_trained_model.pth'

# Save the state dictionary of the final_model
# This saves only the learned parameters, not the model architecture
torch.save(final_model.state_dict(), final_model_save_path)
print(f"\nFinal trained model saved to: {final_model_save_path}")


# Load best model
final_model.load_state_dict(torch.load("best_efficientnet.pth"))
final_model.eval()

# Evaluate on test set
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = final_model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="weighted")
print(f"\nTest Results:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Classify a single X-ray example
image_path = os.path.join(test_dir, "Class1", os.listdir(os.path.join(test_dir, "Class1"))[0])
image = Image.open(image_path).convert("RGB")
input_tensor = data_transforms['test'](image).unsqueeze(0).to(device)
final_model.eval()
with torch.no_grad():
    output = final_model(input_tensor)
    _, predicted = torch.max(output, 1)
    class_idx = predicted.item()
    class_name = class_names[class_idx]
    print(f"\nPredicted class for {image_path}: {class_name}")

# Export to TorchScript
example_input = torch.rand(1, 3, 300, 300).to(device)
try:
    traced_model = torch.jit.trace(final_model, example_input, strict=False)
    output_path = '/content/drive/My Drive/efficientnet_b3_classifier.pt'
    torch.jit.save(traced_model, output_path)
    print(f"\nModel exported to '{output_path}'")
except RuntimeError as e:
    print(f"Tracing failed: {e}")
    output_path = '/content/drive/My Drive/efficientnet_b3_state_dict.pth'
    torch.save(final_model.state_dict(), output_path)
    print(f"\nTracing failed, saved state dict instead to '{output_path}'")

Mounted at /content/drive
xray_data_path set to: /content/drive/My Drive/xray_data/data
01: 70 images found
02: 70 images found
03: 111 images found
Using device: cuda
Class Class1: Train=42, Val=14, Test=14
Class Class2: Train=42, Val=14, Test=14


[I 2025-07-16 20:42:14,552] A new study created in memory with name: no-name-482b5ed7-059c-4fe8-a9f7-b271531c9958


Class Class3: Train=66, Val=22, Test=23
train/Class1: 63 files
train/Class2: 63 files
train/Class3: 94 files
val/Class1: 27 files
val/Class2: 27 files
val/Class3: 41 files
test/Class1: 28 files
test/Class2: 28 files
test/Class3: 44 files
Data split complete!
Train dataset size: 220
Val dataset size: 95
Test dataset size: 100
Class mapping: {'Class1': 0, 'Class2': 1, 'Class3': 2}


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth
100%|██████████| 47.1M/47.1M [00:01<00:00, 29.7MB/s]


Loaded pretrained weights for efficientnet-b3
Epoch 1/13, Train Loss: 1.0475, Val Loss: 1.0607, Val Acc: 37.89%
Epoch 2/13, Train Loss: 0.8939, Val Loss: 1.0044, Val Acc: 44.21%
Epoch 3/13, Train Loss: 0.7320, Val Loss: 0.9284, Val Acc: 63.16%
Epoch 4/13, Train Loss: 0.5962, Val Loss: 0.8391, Val Acc: 75.79%
Epoch 5/13, Train Loss: 0.4080, Val Loss: 0.7343, Val Acc: 82.11%
Epoch 6/13, Train Loss: 0.3387, Val Loss: 0.6337, Val Acc: 82.11%
Epoch 7/13, Train Loss: 0.2324, Val Loss: 0.5474, Val Acc: 82.11%
Epoch 8/13, Train Loss: 0.1760, Val Loss: 0.4666, Val Acc: 85.26%
Epoch 9/13, Train Loss: 0.1390, Val Loss: 0.4275, Val Acc: 84.21%
Epoch 10/13, Train Loss: 0.0938, Val Loss: 0.3760, Val Acc: 86.32%
Epoch 11/13, Train Loss: 0.0860, Val Loss: 0.3252, Val Acc: 88.42%
Epoch 12/13, Train Loss: 0.0903, Val Loss: 0.2852, Val Acc: 89.47%


...

Epoch 15/15, Train Loss: 0.0570, Val Loss: 0.7200, Val Acc: 77.89%
Loaded pretrained weights for efficientnet-b3
Epoch 1/10, Train Loss: 1.0123, Val Loss: 1.1024, Val Acc: 32.63%
Epoch 2/10, Train Loss: 0.7423, Val Loss: 0.9958, Val Acc: 45.26%
Epoch 3/10, Train Loss: 0.5388, Val Loss: 0.8522, Val Acc: 56.84%
Epoch 4/10, Train Loss: 0.3694, Val Loss: 0.6917, Val Acc: 69.47%
Epoch 5/10, Train Loss: 0.2536, Val Loss: 0.5510, Val Acc: 80.00%
Epoch 6/10, Train Loss: 0.1847, Val Loss: 0.4301, Val Acc: 86.32%
Epoch 7/10, Train Loss: 0.1091, Val Loss: 0.3513, Val Acc: 90.53%
Epoch 8/10, Train Loss: 0.1215, Val Loss: 0.2963, Val Acc: 91.58%
Epoch 9/10, Train Loss: 0.1283, Val Loss: 0.2429, Val Acc: 91.58%


[I 2025-07-16 21:45:50,627] Trial 18 finished with value: 94.73684210526316 and parameters: {'lr': 7.793658073169e-05, 'batch_size': 16, 'dropout_rate': 0.3315129292996314, 'num_epochs': 10}. Best is trial 12 with value: 100.0.


Epoch 10/10, Train Loss: 0.0783, Val Loss: 0.2081, Val Acc: 94.74%
Loaded pretrained weights for efficientnet-b3
Epoch 1/13, Train Loss: 0.9728, Val Loss: 12050.1504, Val Acc: 50.53%
Epoch 2/13, Train Loss: 0.7898, Val Loss: 16975.4431, Val Acc: 42.11%
Epoch 3/13, Train Loss: 0.5288, Val Loss: 162664.8333, Val Acc: 43.16%
Epoch 4/13, Train Loss: 0.5601, Val Loss: 352985.2161, Val Acc: 28.42%
Epoch 5/13, Train Loss: 0.5056, Val Loss: 16597.0884, Val Acc: 28.42%


[I 2025-07-16 21:47:35,726] Trial 19 finished with value: 50.526315789473685 and parameters: {'lr': 0.004773467775767406, 'batch_size': 16, 'dropout_rate': 0.4453952322380778, 'num_epochs': 13}. Best is trial 12 with value: 100.0.


Epoch 6/13, Train Loss: 0.2257, Val Loss: 3952.9656, Val Acc: 28.42%
Early stopping at epoch 6
Best hyperparameters: {'lr': 0.0005317546126948767, 'batch_size': 16, 'dropout_rate': 0.3620640042753704, 'num_epochs': 12}
Best validation accuracy: 100.0
Loaded pretrained weights for efficientnet-b3
Epoch 1/12, Train Loss: 0.6547, Val Loss: 0.7018, Val Acc: 65.26%
Epoch 2/12, Train Loss: 0.1820, Val Loss: 0.8637, Val Acc: 71.58%
Epoch 3/12, Train Loss: 0.2748, Val Loss: 1.5527, Val Acc: 63.16%
Epoch 4/12, Train Loss: 0.2123, Val Loss: 0.1331, Val Acc: 93.68%
Epoch 5/12, Train Loss: 0.0880, Val Loss: 0.0948, Val Acc: 96.84%
Epoch 6/12, Train Loss: 0.0829, Val Loss: 0.1146, Val Acc: 96.84%
Epoch 7/12, Train Loss: 0.0579, Val Loss: 0.0529, Val Acc: 97.89%
Epoch 8/12, Train Loss: 0.0912, Val Loss: 0.0986, Val Acc: 93.68%
Epoch 9/12, Train Loss: 0.1530, Val Loss: 0.3031, Val Acc: 86.32%
Epoch 10/12, Train Loss: 0.0919, Val Loss: 0.0711, Val Acc: 97.89%
Epoch 11/12, Train Loss: 0.0421, Val Loss:

## Same code as above skipping train test split if exist

In [ ]:
import os
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from efficientnet_pytorch import EfficientNet
import optuna
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import torch.nn.functional as F
import seaborn as sns # Added for heatmap

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set xray_data_path to your new combined_full_3_classes directory
xray_data_path = '/content/drive/My Drive/xray_data/combined_full_3_classes'
print(f"xray_data_path set to: {xray_data_path}")

# IMPORTANT: These must EXACTLY match your *renamed* actual folder names
# As discussed, for ImageFolder's default alphabetical mapping:
# 'covid' -> 0, 'normal' -> 1, 'pneumonia' -> 2
# If you actually renamed your folders to e.g., '0_pneumonia', '1_normal', '2_covid',
# then original_classes and MODEL_CLASS_LABELS should reflect that.
original_classes = ["pneumonia", "normal", "covid"] # Assuming these are your actual folder names
new_class_names = ["pneumonia", "normal", "covid"] # This is how you want them named internally

# Model's output class labels (mapping from model's predicted index to human-readable string)
# Adjust this based on how ImageFolder actually maps (alphabetical by folder name)
# If original_classes are "covid", "normal", "pneumonia", then ImageFolder maps:
# 0: "covid", 1: "normal", 2: "pneumonia"
# If original_classes are "pneumonia", "normal", "covid", and you renamed the folders accordingly
# (e.g., "0_pneumonia", "1_normal", "2_covid"), then this mapping is correct.
MODEL_CLASS_LABELS = {
    0: "pneumonia",
    1: "normal",
    2: "covid"
}
NUM_MODEL_OUTPUT_CLASSES = len(MODEL_CLASS_LABELS) # Should be 3 for this classification task

# --- START OF NEW ADDITION ---
# Global flag to control recreation of train/val/test splits
RECREATE_TRAIN_TEST_SPLIT = False # Set to True to force recreation, False to reuse existing
# --- END OF NEW ADDITION ---


# Verify data existence with new subfolder names
print("\nVerifying raw data existence in new path:")
for subfolder in original_classes:
    subfolder_path = os.path.join(xray_data_path, subfolder)
    if not os.path.exists(subfolder_path):
        raise FileNotFoundError(f"Subfolder {subfolder_path} not found. Please ensure these exact subfolders exist.")
    images = [f for f in os.listdir(subfolder_path) if f.lower().endswith((".jpg", ".png", ".jpeg"))]
    print(f"{subfolder}: {len(images)} images found")
    if not images:
        raise FileNotFoundError(f"No images found in {subfolder_path}. This class will be empty.")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define data transforms with balanced augmentation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((300, 300)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(15),
        transforms.RandomAffine(degrees=5, translate=(0.05, 0.05)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((300, 300)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((300, 300)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

# Split data into train, val, test (60-20-20)
base_dir = xray_data_path
train_dir = os.path.join(base_dir, "train_split")
val_dir = os.path.join(base_dir, "val_split")
test_dir = os.path.join(base_dir, "test_split")

# --- START OF NEW ADDITION ---
# Check if split directories exist
split_dirs_exist = all(os.path.exists(os.path.join(d, c)) for d in [train_dir, val_dir, test_dir] for c in new_class_names)

if RECREATE_TRAIN_TEST_SPLIT or not split_dirs_exist:
    print("\nRecreating train/val/test splits...")
    # Clean up existing split directories if they exist and we're recreating
    if split_dirs_exist:
        print("Removing existing split directories...")
        for dir_path in [train_dir, val_dir, test_dir]:
            if os.path.exists(dir_path):
                shutil.rmtree(dir_path)

    # Create new directories
    print("Creating new data split directories...")
    for dir_path in [train_dir, val_dir, test_dir]:
        for class_name in new_class_names:
            os.makedirs(os.path.join(dir_path, class_name), exist_ok=True)
    print("Directories created.")

    # Split and copy images
    print("\nSplitting and copying images...")
    RANDOM_SEED = 23
    for old_class_folder, new_class_folder in zip(original_classes, new_class_names):
        class_path = os.path.join(base_dir, old_class_folder)
        images = [f for f in os.listdir(class_path) if f.lower().endswith((".jpg", ".png", ".jpeg"))]
        image_paths = [os.path.join(class_path, img) for img in images]

        train_imgs, temp_imgs = train_test_split(image_paths, test_size=0.4, random_state=RANDOM_SEED)
        val_imgs, test_imgs = train_test_split(temp_imgs, test_size=0.5, random_state=RANDOM_SEED)

        for img in train_imgs:
            dest = os.path.join(train_dir, new_class_folder, os.path.basename(img))
            shutil.copy(img, dest)
        for img in val_imgs:
            dest = os.path.join(val_dir, new_class_folder, os.path.basename(img))
            shutil.copy(img, dest)
        for img in test_imgs:
            dest = os.path.join(test_dir, new_class_folder, os.path.basename(img))
            shutil.copy(img, dest)

        print(f"Class {new_class_folder}: Train={len(train_imgs)}, Val={len(val_imgs)}, Test={len(test_imgs)}")
    print("Data split complete!")
else:
    print("\nExisting train/val/test splits found and RECREATE_TRAIN_TEST_SPLIT is False. Reusing existing data.")

# Verify files were copied (or exist if reusing)
print("\nVerifying copied file counts:")
for dir_path, split_name in [(train_dir, "train"), (val_dir, "val"), (test_dir, "test")]:
    for class_name in new_class_names:
        class_dir = os.path.join(dir_path, class_name)
        num_files = len([f for f in os.listdir(class_dir) if f.lower().endswith((".jpg", ".png", ".jpeg"))])
        print(f"- {split_name}/{class_name}: {num_files} files")
        if num_files == 0:
            print(f"  WARNING: No files found in {class_dir}. This might indicate an issue with initial data or copying.")
# --- END OF NEW ADDITION ---


# Load datasets
train_dataset = ImageFolder(root=train_dir, transform=data_transforms['train'])
val_dataset = ImageFolder(root=val_dir, transform=data_transforms['val'])
test_dataset = ImageFolder(root=test_dir, transform=data_transforms['test'])

print(f"\nLoaded Datasets:")
print(f"Train dataset size: {len(train_dataset)}")
print(f"Val dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")
print("ImageFolder class mapping:", train_dataset.class_to_idx) # This will show the actual mapping ImageFolder created

# Define the model with SiLU instead of Swish for TorchScript compatibility
def create_model(dropout_rate=0.35):
    model = EfficientNet.from_pretrained('efficientnet-b3')
    # Replace Swish with SiLU
    for module in model.modules():
        if hasattr(module, '_swish'):
            module._swish = nn.SiLU(inplace=True)
    num_features = model._fc.in_features
    model._fc = nn.Sequential(
        nn.Dropout(p=dropout_rate),
        nn.Linear(num_features, NUM_MODEL_OUTPUT_CLASSES)
    )
    return model.to(device)

# Training function with early stopping and scheduler
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience=5):
    best_val_acc = 0.0
    patience_counter = 0
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        train_loss = running_loss / len(train_loader)

        model.eval()
        correct = 0
        total = 0
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(val_loader)
        val_acc = 100 * correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        scheduler.step(val_acc)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_efficientnet.pth")
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
    return best_val_acc

# Objective function for Optuna with updated suggestions
def objective(trial):
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    dropout_rate = trial.suggest_float("dropout_rate", 0.3, 0.5)
    num_epochs = trial.suggest_int("num_epochs", 5, 15)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    model = create_model(dropout_rate)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=5e-5)

    val_acc = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)
    return val_acc

# Optimize hyperparameters with Optuna
print("\nStarting Optuna hyperparameter optimization...")
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)
print("\n--- Optuna Optimization Results ---")
print("Best hyperparameters:", study.best_params)
print("Best validation accuracy:", study.best_value)

# Train final model with best parameters
print("\nTraining final model with best hyperparameters...")
best_params = study.best_params
train_loader = DataLoader(train_dataset, batch_size=best_params["batch_size"], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=best_params["batch_size"], shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=best_params["batch_size"], shuffle=False)

final_model = create_model(best_params["dropout_rate"])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(final_model.parameters(), lr=best_params["lr"], weight_decay=5e-5)
train_model(final_model, train_loader, val_loader, criterion, optimizer, best_params["num_epochs"])

# Define the path to save the final model's state dictionary
final_model_save_path = '/content/drive/My Drive/xray_data/kaggle_data_efficientnet_trained_model.pth'
torch.save(final_model.state_dict(), final_model_save_path)
print(f"\nFinal trained model state dictionary saved to: {final_model_save_path}")

# Load best model (from the one saved during early stopping)
final_model.load_state_dict(torch.load("best_efficientnet.pth"))
final_model.eval()
print("\nLoaded best model for evaluation.")

# Evaluate on test set
print("\n--- Evaluating Model on Test Set ---")
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = final_model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="weighted", zero_division=0)

print(f"\nTest Results:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision (weighted): {precision:.4f}")
print(f"Recall (weighted): {recall:.4f}")
print(f"F1-Score (weighted): {f1:.4f}")

# Generate and print a detailed classification report
print("\nClassification Report (Precision, Recall, F1-Score per class):")
target_names_for_report = [MODEL_CLASS_LABELS[i] for i in sorted(MODEL_CLASS_LABELS.keys())]
print(classification_report(all_labels, all_preds, target_names=target_names_for_report, zero_division=0))

# Generate and plot the Confusion Matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(all_labels, all_preds)
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names_for_report,
            yticklabels=target_names_for_report)
plt.title('Confusion Matrix for X-ray Classification (EfficientNet)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.show()

# Classify a single X-ray example
print("\n--- Single Image Classification Example ---")
try:
    example_class_dir = os.path.join(test_dir, new_class_names[0])
    example_image_filename = os.listdir(example_class_dir)[0]
    image_path = os.path.join(example_class_dir, example_image_filename)
except (IndexError, FileNotFoundError):
    print(f"Could not find an example image in {os.path.join(test_dir, new_class_names[0])}. Please ensure images were copied there.")
    image_path = None

if image_path and os.path.exists(image_path):
    print(f"Classifying example image: {image_path}")
    image = Image.open(image_path).convert("RGB")
    input_tensor = data_transforms['test'](image).unsqueeze(0).to(device)

    final_model.eval()
    with torch.no_grad():
        output = final_model(input_tensor)
        probabilities = F.softmax(output, dim=1)[0]
        _, predicted = torch.max(output, 1)

        class_idx = predicted.item()
        class_name = MODEL_CLASS_LABELS[class_idx]
        predicted_score = round(probabilities[class_idx].item() * 100, 2)

        print(f"Predicted class for '{os.path.basename(image_path)}': {class_name} ({predicted_score}%)")
else:
    print("Skipping single image classification as no example image path was found or exists.")

# Export to TorchScript
example_input = torch.rand(1, 3, 300, 300).to(device)
try:
    traced_model = torch.jit.trace(final_model, example_input, strict=False)
    output_path = '/content/drive/My Drive/xray_data/kaggle_data_efficientnet_b3_classifier.pt'
    torch.jit.save(traced_model, output_path)
    print(f"\nModel exported to '{output_path}'")
except RuntimeError as e:
    print(f"Tracing failed: {e}")
    output_path = '/content/drive/My Drive/xray_data/kaggle_data_efficientnet_b3_state_dict.pth'
    torch.save(final_model.state_dict(), output_path)
    print(f"\nTracing failed, saved state dict instead to '{output_path}'")

## Same Code with with Vgg16 Instead of EfficientNet

In [ ]:
import os
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models import vgg16_bn # Changed from efficientnet_pytorch import EfficientNet
import optuna
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import torch.nn.functional as F
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

xray_data_path = '/content/drive/My Drive/xray_data/combined_full_3_classes'
print(f"xray_data_path set to: {xray_data_path}")

original_classes = ["pneumonia", "normal", "covid"]
new_class_names = ["pneumonia", "normal", "covid"]

MODEL_CLASS_LABELS = {
    0: "pneumonia",
    1: "normal",
    2: "covid"
}
NUM_MODEL_OUTPUT_CLASSES = len(MODEL_CLASS_LABELS)

RECREATE_TRAIN_TEST_SPLIT = False

print("\nVerifying raw data existence in new path:")
for subfolder in original_classes:
    subfolder_path = os.path.join(xray_data_path, subfolder)
    if not os.path.exists(subfolder_path):
        raise FileNotFoundError(f"Subfolder {subfolder_path} not found. Please ensure these exact subfolders exist.")
    images = [f for f in os.listdir(subfolder_path) if f.lower().endswith((".jpg", ".png", ".jpeg"))]
    print(f"{subfolder}: {len(images)} images found")
    if not images:
        raise FileNotFoundError(f"No images found in {subfolder_path}. This class will be empty.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)), # Changed from 300x300 for VGG16 standard input size
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(15),
        transforms.RandomAffine(degrees=5, translate=(0.05, 0.05)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)), # Changed from 300x300
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)), # Changed from 300x300
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

base_dir = xray_data_path
train_dir = os.path.join(base_dir, "train_split")
val_dir = os.path.join(base_dir, "val_split")
test_dir = os.path.join(base_dir, "test_split")

split_dirs_exist = all(os.path.exists(os.path.join(d, c)) for d in [train_dir, val_dir, test_dir] for c in new_class_names)

if RECREATE_TRAIN_TEST_SPLIT or not split_dirs_exist:
    print("\nRecreating train/val/test splits...")
    if split_dirs_exist:
        print("Removing existing split directories...")
        for dir_path in [train_dir, val_dir, test_dir]:
            if os.path.exists(dir_path):
                shutil.rmtree(dir_path)

    print("Creating new data split directories...")
    for dir_path in [train_dir, val_dir, test_dir]:
        for class_name in new_class_names:
            os.makedirs(os.path.join(dir_path, class_name), exist_ok=True)
    print("Directories created.")

    print("\nSplitting and copying images...")
    RANDOM_SEED = 23
    for old_class_folder, new_class_folder in zip(original_classes, new_class_names):
        class_path = os.path.join(base_dir, old_class_folder)
        images = [f for f in os.listdir(class_path) if f.lower().endswith((".jpg", ".png", ".jpeg"))]
        image_paths = [os.path.join(class_path, img) for img in images]

        train_imgs, temp_imgs = train_test_split(image_paths, test_size=0.4, random_state=RANDOM_SEED)
        val_imgs, test_imgs = train_test_split(temp_imgs, test_size=0.5, random_state=RANDOM_SEED)

        for img in train_imgs:
            dest = os.path.join(train_dir, new_class_folder, os.path.basename(img))
            shutil.copy(img, dest)
        for img in val_imgs:
            dest = os.path.join(val_dir, new_class_folder, os.path.basename(img))
            shutil.copy(img, dest)
        for img in test_imgs:
            dest = os.path.join(test_dir, new_class_folder, os.path.basename(img))
            shutil.copy(img, dest)

        print(f"Class {new_class_folder}: Train={len(train_imgs)}, Val={len(val_imgs)}, Test={len(test_imgs)}")
    print("Data split complete!")
else:
    print("\nExisting train/val/test splits found and RECREATE_TRAIN_TEST_SPLIT is False. Reusing existing data.")

print("\nVerifying copied file counts:")
for dir_path, split_name in [(train_dir, "train"), (val_dir, "val"), (test_dir, "test")]:
    for class_name in new_class_names:
        class_dir = os.path.join(dir_path, class_name)
        num_files = len([f for f in os.listdir(class_dir) if f.lower().endswith((".jpg", ".png", ".jpeg"))])
        print(f"- {split_name}/{class_name}: {num_files} files")
        if num_files == 0:
            print(f"  WARNING: No files found in {class_dir}. This might indicate an issue with initial data or copying.")

train_dataset = ImageFolder(root=train_dir, transform=data_transforms['train'])
val_dataset = ImageFolder(root=val_dir, transform=data_transforms['val'])
test_dataset = ImageFolder(root=test_dir, transform=data_transforms['test'])

print(f"\nLoaded Datasets:")
print(f"Train dataset size: {len(train_dataset)}")
print(f"Val dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")
print("ImageFolder class mapping:", train_dataset.class_to_idx)

def create_model(dropout_rate=0.5): # Changed default dropout for VGG, often higher
    model = vgg16_bn(pretrained=True) # Changed EfficientNet.from_pretrained to vgg16

    # Unfreeze last 2 convolutional blocks (example: last 8 layers)
    for name, param in list(model.features.named_parameters())[-8:]:
        param.requires_grad = True

    num_features = model.classifier[6].in_features
    model.classifier[6] = nn.Sequential(
        nn.Dropout(p=dropout_rate),
        nn.Linear(num_features, NUM_MODEL_OUTPUT_CLASSES)
    )
    return model.to(device)

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience=5):
    best_val_acc = 0.0
    patience_counter = 0
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        train_loss = running_loss / len(train_loader)

        model.eval()
        correct = 0
        total = 0
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(val_loader)
        val_acc = 100 * correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        scheduler.step(val_acc)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_vgg16.pth") # Changed filename
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
    return best_val_acc

# Objective function for Optuna with updated suggestions
def objective(trial):
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    dropout_rate = trial.suggest_float("dropout_rate", 0.3, 0.5)
    num_epochs = trial.suggest_int("num_epochs", 5, 15)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    model = create_model(dropout_rate)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=5e-5)

    val_acc = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)
    return val_acc

print("\nStarting Optuna hyperparameter optimization...")
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)
print("\n--- Optuna Optimization Results ---")
print("Best hyperparameters:", study.best_params)
print("Best validation accuracy:", study.best_value)

print("\nTraining final model with best hyperparameters...")
best_params = study.best_params
train_loader = DataLoader(train_dataset, batch_size=best_params["batch_size"], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=best_params["batch_size"], shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=best_params["batch_size"], shuffle=False)

final_model = create_model(best_params["dropout_rate"])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(final_model.parameters(), lr=best_params["lr"], weight_decay=5e-5)
train_model(final_model, train_loader, val_loader, criterion, optimizer, best_params["num_epochs"])

final_model_save_path = '/content/drive/My Drive/xray_data/kaggle_data_vgg16_trained_model.pth' # Changed filename
torch.save(final_model.state_dict(), final_model_save_path)
print(f"\nFinal trained model state dictionary saved to: {final_model_save_path}")

final_model.load_state_dict(torch.load("best_vgg16.pth")) # Changed filename
final_model.eval()
print("\nLoaded best model for evaluation.")

print("\n--- Evaluating Model on Test Set ---")
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = final_model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="weighted", zero_division=0)

print(f"\nTest Results:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision (weighted): {precision:.4f}")
print(f"Recall (weighted): {recall:.4f}")
print(f"F1-Score (weighted): {f1:.4f}")

print("\nClassification Report (Precision, Recall, F1-Score per class):")
target_names_for_report = [MODEL_CLASS_LABELS[i] for i in sorted(MODEL_CLASS_LABELS.keys())]
print(classification_report(all_labels, all_preds, target_names=target_names_for_report, zero_division=0))

print("\nConfusion Matrix:")
cm = confusion_matrix(all_labels, all_preds)
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names_for_report,
            yticklabels=target_names_for_report)
plt.title('Confusion Matrix for X-ray Classification (VGG16)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.show()

print("\n--- Single Image Classification Example ---")
try:
    example_class_dir = os.path.join(test_dir, new_class_names[0])
    example_image_filename = os.listdir(example_class_dir)[0]
    image_path = os.path.join(example_class_dir, example_image_filename)
except (IndexError, FileNotFoundError):
    print(f"Could not find an example image in {os.path.join(test_dir, new_class_names[0])}. Please ensure images were copied there.")
    image_path = None

if image_path and os.path.exists(image_path):
    print(f"Classifying example image: {image_path}")
    image = Image.open(image_path).convert("RGB")
    input_tensor = data_transforms['test'](image).unsqueeze(0).to(device)

    final_model.eval()
    with torch.no_grad():
        output = final_model(input_tensor)
        probabilities = F.softmax(output, dim=1)[0]
        _, predicted = torch.max(output, 1)

        class_idx = predicted.item()
        class_name = MODEL_CLASS_LABELS[class_idx]
        predicted_score = round(probabilities[class_idx].item() * 100, 2)

        print(f"Predicted class for '{os.path.basename(image_path)}': {class_name} ({predicted_score}%)")
else:
    print("Skipping single image classification as no example image path was found or exists.")

example_input = torch.rand(1, 3, 224, 224).to(device) # Changed input size for VGG16
try:
    traced_model = torch.jit.trace(final_model, example_input, strict=False)
    output_path = '/content/drive/My Drive/xray_data/kaggle_data_vgg16_classifier.pt' # Changed filename
    torch.jit.save(traced_model, output_path)
    print(f"\nModel exported to '{output_path}'")
except RuntimeError as e:
    print(f"Tracing failed: {e}")
    output_path = '/content/drive/My Drive/xray_data/kaggle_data_vgg16_state_dict.pth' # Changed filename
    torch.save(final_model.state_dict(), output_path)
    print(f"\nTracing failed, saved state dict instead to '{output_path}'")

Mounted at /content/drive
xray_data_path set to: /content/drive/My Drive/xray_data/combined_full_3_classes

Verifying raw data existence in new path:
pneumonia: 5618 images found
normal: 11775 images found
covid: 3616 images found
Using device: cuda

Existing train/val/test splits found and RECREATE_TRAIN_TEST_SPLIT is False. Reusing existing data.

Verifying copied file counts:
- train/pneumonia: 4073 files
- train/normal: 7085 files
- train/covid: 3059 files
- val/pneumonia: 1588 files
- val/normal: 2368 files
- val/covid: 1312 files
- test/pneumonia: 1597 files
- test/normal: 2366 files
- test/covid: 1310 files


[I 2025-07-22 17:25:39,891] A new study created in memory with name: no-name-9f9cf1d6-95bb-4e75-8043-ffac98240d65



Loaded Datasets:
Train dataset size: 14217
Val dataset size: 5268
Test dataset size: 5273
ImageFolder class mapping: {'covid': 0, 'normal': 1, 'pneumonia': 2}

Starting Optuna hyperparameter optimization...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 176MB/s]


Epoch 1/15, Train Loss: 1.8538, Val Loss: 2.0555, Val Acc: 71.22%
Epoch 2/15, Train Loss: 2.1443, Val Loss: 2.6618, Val Acc: 70.05%
Epoch 3/15, Train Loss: 2.3785, Val Loss: 0.8718, Val Acc: 82.90%
Epoch 4/15, Train Loss: 2.3806, Val Loss: 1.4260, Val Acc: 81.26%
Epoch 5/15, Train Loss: 2.3508, Val Loss: 1.2348, Val Acc: 83.24%
Epoch 6/15, Train Loss: 2.4327, Val Loss: 1.9542, Val Acc: 78.13%
Epoch 7/15, Train Loss: 2.4497, Val Loss: 2.4096, Val Acc: 75.36%
Epoch 8/15, Train Loss: 2.5136, Val Loss: 1.3944, Val Acc: 81.51%
Epoch 9/15, Train Loss: 2.0690, Val Loss: 1.2389, Val Acc: 81.06%


...

Epoch 5/5, Train Loss: 0.4671, Val Loss: 0.4050, Val Acc: 83.12%
Epoch 1/7, Train Loss: 0.5555, Val Loss: 0.5867, Val Acc: 72.74%
Epoch 2/7, Train Loss: 0.4723, Val Loss: 0.4465, Val Acc: 79.95%
Epoch 3/7, Train Loss: 0.4663, Val Loss: 0.4322, Val Acc: 80.94%
Epoch 4/7, Train Loss: 0.4554, Val Loss: 0.4716, Val Acc: 78.51%
Epoch 5/7, Train Loss: 0.4633, Val Loss: 0.3921, Val Acc: 83.45%
Epoch 6/7, Train Loss: 0.4524, Val Loss: 0.4463, Val Acc: 80.30%
Epoch 7/7, Train Loss: 0.4534, Val Loss: 0.3809, Val Acc: 83.66%


[I 2025-07-22 20:47:21,009] Trial 2 finished with value: 83.65603644646924 and parameters: {'lr': 0.00035592573515080866, 'batch_size': 16, 'dropout_rate': 0.3453415152996004, 'num_epochs': 7}. Best is trial 1 with value: 83.76993166287016.


Epoch 1/9, Train Loss: 0.8068, Val Loss: 0.3639, Val Acc: 85.29%
Epoch 2/9, Train Loss: 0.8360, Val Loss: 0.6454, Val Acc: 78.21%
Epoch 3/9, Train Loss: 0.8613, Val Loss: 1.2454, Val Acc: 70.48%
Epoch 4/9, Train Loss: 0.9010, Val Loss: 0.7432, Val Acc: 78.21%
Epoch 5/9, Train Loss: 0.8073, Val Loss: 0.5761, Val Acc: 80.56%


[I 2025-07-22 21:18:54,395] Trial 3 finished with value: 85.28853454821564 and parameters: {'lr': 0.00126475598134077, 'batch_size': 8, 'dropout_rate': 0.46585118400450104, 'num_epochs': 9}. Best is trial 3 with value: 85.28853454821564.


Epoch 6/9, Train Loss: 0.7040, Val Loss: 0.4908, Val Acc: 81.11%
Early stopping at epoch 6
Epoch 1/14, Train Loss: 0.9488, Val Loss: 0.7803, Val Acc: 60.95%
Epoch 2/14, Train Loss: 0.7488, Val Loss: 0.7037, Val Acc: 64.65%
Epoch 3/14, Train Loss: 0.6569, Val Loss: 0.6392, Val Acc: 69.17%
Epoch 4/14, Train Loss: 0.6167, Val Loss: 0.6179, Val Acc: 70.37%
Epoch 5/14, Train Loss: 0.5794, Val Loss: 0.6035, Val Acc: 71.07%
Epoch 6/14, Train Loss: 0.5675, Val Loss: 0.5911, Val Acc: 71.75%
Epoch 7/14, Train Loss: 0.5450, Val Loss: 0.5695, Val Acc: 73.37%
Epoch 8/14, Train Loss: 0.5347, Val Loss: 0.5596, Val Acc: 73.94%
